In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [ ]:
#import data
main_data = pd.read_csv("../input/craigslist-carstrucks-data/vehicles.csv")
main_data.shape

### Inspect data features

In [ ]:
main_data.head()

In [ ]:
main_data.info()

# Possible EDA 
1. Informative columns for eda include
    * Region
    * price 
    * year
    * manufacturer
    * model
    * Condition
    * Cylinders
    * fuel
    * odometer
    * Title status
    * Transmission
    * Drive 
    * size
    * type
    * color
    * state - lat and logitude only describe a state

In [ ]:
#create a eda data based on listed data features
eda_data = main_data.drop(columns=["url", "region_url", "vin", "image_url", "description", "county"])

In [ ]:
eda_data.head()

In [ ]:
eda_data.info()

### Preprocess data for EDA


In [ ]:
"""numerical_data = eda_data.select_dtypes(include=['int64', 'float64']).columns
categorical_data = eda_data.select_dtypes(include=['object']).columns

#replace missing data
eda_data[numerical_data] = eda_data[numerical_data].dropna()
eda_data[categorical_data] = eda_data[categorical_data].fillna("missing")"""

In [ ]:
#handle the cylinder column
eda_data["cylinders"] = eda_data["cylinders"].str[0]

In [ ]:
eda_data.head()

In [ ]:
#create car age column
eda_data["car_age"] = (2020 - eda_data["year"])

In [ ]:
eda_data.head()

### start analysis from region


In [ ]:
listed_cars = eda_data.groupby(['region'])['id'].count().to_frame(name="cars_count")
listed_cars = listed_cars.sort_values(by = ["cars_count"], ascending = False).reset_index()
print("datashape is {}".format(listed_cars.shape))

#plot the data
fig = px.bar(listed_cars, x=listed_cars.region, y=listed_cars.cars_count, title="Used cars per region")
fig.show()

### **Observation**
* There ia a big difference in the distribution of listed cars in each region with columbus(3296) being the highest and fort smith,AR(1) being the lowest

### **inference**
* There could be a number of possible reasons why this is the case, one of which maybe be related to Columbus being the most populated city in the state of Ohio

In [ ]:
listed_cars_price = eda_data.groupby(['region'])['price'].sum().to_frame(name="cars_price")
listed_cars_price = listed_cars_price.sort_values(by = ["cars_price"], ascending = False).reset_index()
print("datashape is {}".format(listed_cars_price.shape))

#plot the data
fig = px.bar(listed_cars_price, x=listed_cars_price.region, y=listed_cars_price.cars_price, title="Used cars per region")
fig.show()

### **observation**
1. regions with high numbers of listed cars are not featured amongst the region with high sum of prices for listed cars. In fact, columbus with the highest number of listed cars appears pretty low on the price sum chart

In [ ]:
listed_cars_price[listed_cars_price['region'] == 'columbus']

### **Inference**
This phenomena can be due to multiple reasons;

1. age of cars in the region
2. Fuel type
3. car condition
4. manufacturer
5. transmission type
6. car model and 
7. car cylinders
8.  

In [ ]:
#age distribution of cars
age_region_distribution = eda_data.groupby(['region'])['car_age'].sum().to_frame(name = 'car_age_sum')
age_region_distribution = age_region_distribution.sort_values(by = ['car_age_sum'], ascending = False).reset_index()


In [ ]:
fig = px.bar(age_region_distribution, x = age_region_distribution.region, y=age_region_distribution.car_age_sum, title=' Sum of age across regions')
fig.show()

### **Observation**
1. columbus which has a high count of listed cars and a low sum of listed car prices is observed to rank 3rd on the car age chart.
2. Pittsburgh also follow the columbus trend, along with other regions
3. flint which ranks 1st on the price list contains pretty young cars ranking 124th on the car age chart



In [ ]:
age_region_distribution[age_region_distribution['region'] == 'flint']

### **Inference**
Older cars are more likely to cost low compared to younger cars

*  inspect further

In [ ]:
eda_data['fuel'].value_counts()

In [ ]:
#fuel_type distribution of cars
fuel_type_distribution = eda_data.groupby(['region', 'fuel'])['fuel'].count().to_frame(name = 'fuel_type_count')
fuel_type_distribution = fuel_type_distribution.sort_values(by = ['fuel_type_count'], ascending = False).reset_index()


In [ ]:
fuel_type_distribution.head()
gas_df = fuel_type_distribution[fuel_type_distribution['fuel'] == 'gas'].reset_index()
diesel_df = fuel_type_distribution[fuel_type_distribution['fuel'] == 'diesel'].reset_index()
other_df = fuel_type_distribution[fuel_type_distribution['fuel'] == 'other'].reset_index()
hybrid_df = fuel_type_distribution[fuel_type_distribution['fuel'] == 'hybrid'].reset_index()
electric_df = fuel_type_distribution[fuel_type_distribution['fuel'] == 'electric'].reset_index()

In [ ]:
gas_df.head()

In [ ]:
#check dominant fuel type

fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=gas_df.region[:50], values=gas_df.fuel_type_count, name="gas"),
              1, 1)
fig.add_trace(go.Pie(labels=diesel_df.region[:50], values=diesel_df.fuel_type_count, name="diesel"),
              1, 2)


# Use `hole` to create a donut-like pie chart
fig.update_traces(hole=.4, hoverinfo="label+percent+name")

fig.update_layout(
    title_text="Fuel type distributions for GAS and DIESEL",
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='GAS', x=0.18, y=0.5, font_size=20, showarrow=False),
                 dict(text='DIESEL', x=0.82, y=0.5, font_size=20, showarrow=False)])
fig.show()

### **Observation**
1. regions that appear high on the listed cars chart and low on the price sum chart appear to mostly make use of gas and diesel as their source of fuel. columbus and pittsburgh are the evidence

### **inference**

2. cars with lesser gas and diesel fuel types cost less than other fuel types

In [ ]:
#check dominant fuel type

fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=electric_df.region[:50], values=electric_df.fuel_type_count, name="electric"),
              1, 1)
fig.add_trace(go.Pie(labels=hybrid_df.region[:50], values=hybrid_df.fuel_type_count, name="hybrid"),
              1, 2)


# Use `hole` to create a donut-like pie chart
fig.update_traces(hole=.4, hoverinfo="label+percent+name")

fig.update_layout(
    title_text="Fuel type distributions for ELECTRIC and HYBRID",
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='ELECTRIC', x=0.18, y=0.5, font_size=20, showarrow=False),
                 dict(text='HYBRID', x=0.82, y=0.5, font_size=20, showarrow=False)])
fig.show()

In [ ]:
#check dominant fuel type

fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=other_df.region[:50], values=other_df.fuel_type_count, name="other"),
              1, 1)


# Use `hole` to create a donut-like pie chart
fig.update_traces(hole=.4, hoverinfo="label+percent+name")

fig.update_layout(
    title_text="Fuel type distributions for OTHERS",
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='OTHERS', x=0.18, y=0.5, font_size=20, showarrow=False)])
fig.show()

### **Observation**
1. the region with highest price rating features on the others fuel type

### **inference**
* cars with scarce furl types currently cost more


Inspect further

In [ ]:
#group data by condition and prices
condition_df = eda_data.groupby(['condition'])['price'].sum().to_frame(name = 'condition_avg_price')
condition_df = condition_df.sort_values(by = ['condition_avg_price'], ascending = False).reset_index()

In [ ]:
fig = px.bar(condition_df, x = 'condition', y = 'condition_avg_price', color = 'condition_avg_price')
fig.show()

### **Observation**
1. surprisingly, the price sum for cars in a good condition is observed to be much more higher than that of excellent and cars that almost new

### **inference**
1. this observation could be due to the share number of good cars that are listed

inspect further

In [ ]:
#group data by condition and count
condition_count_df = eda_data.groupby(['condition'])['condition'].count().to_frame(name = 'count')
condition_count_df = condition_count_df.sort_values(by = ['count'], ascending = False).reset_index()

In [ ]:
fig = px.bar(condition_count_df, x = 'condition', y = 'count', color = 'count')
fig.show()

### **Observation**
1. the number of excellent cars actually surpasses that of good cars by almost 20000 

#### **Inference**
* which implies that, good cars receive better prices than excellent cars
* The transmission type, cylinder_type and title status may affect these figures. 

inspect further

# EDA still going on